# Day 5b - Agent Deployment (LangChain可本地运行版)

🚀 **将 Agent 部署为独立服务 (LangServe)**

欢迎来到 Kaggle 5-day Agents 课程的最后一天！

在之前的课程中，你已经学会了如何构建、评估以及让 Agent 互相通信。现在，我们将迈出最后一步：**将你的 Agent 部署到生产环境**。

## 💡 为什么要部署？

你构建了一个很棒的 AI Agent，但它只存在于你的 Notebook 里。当你关闭 Notebook 时，它就停止工作了。你的队友无法访问它，用户也无法与它交互。

部署可以将你的 Agent 转化为一个**独立运行的服务 (Service)**，它可以：
1.  **7x24小时运行**：随时响应请求。
2.  **被外部访问**：前端网页、移动应用或其他 Agent 都可以调用它。
3.  **持久化记忆**：即使用户刷新了页面，对话历史依然保留。

## 🔄 LangChain 改写说明：从 Cloud 到 Localhost

原教程演示了如何将 Agent 部署到 Google Vertex AI Agent Engine。为了让你能够在本地完整体验“构建-部署-调用”的全流程，我们将使用 **LangServe** 替代云平台。

| 功能/概念 | 原教程 (Google ADK & Vertex AI) | 本教程 (LangChain & LangServe) | 说明 |
| :--- | :--- | :--- | :--- |
| **部署目标** | Vertex AI Agent Engine (云端全托管) | **LangServe** (本地 FastAPI 服务) | 这是 LangChain 官方推荐的生产级部署方案，支持流式输出和 API 文档。 |
| **部署工具** | `adk deploy` CLI | `uv run server.py` | 我们将编写一个 Python 脚本来启动 Web 服务器。 |
| **客户端 SDK** | `vertexai.agent_engines` | `RemoteRunnable` | LangChain 提供的客户端工具，可以像调用本地函数一样调用远程 API。 |
| **长时记忆** | Vertex AI Memory Bank | **LangGraph Checkpointer** | 使用内存或数据库 (SQLite) 保存对话状态，模拟“记忆库”功能。 |

## 🎯 本教程学习目标

✅ **构建生产就绪的 Agent**：使用 LangGraph 封装业务逻辑。
✅ **创建部署代码**：编写 `server.py`，使用 LangServe 暴露 API。
✅ **本地模拟部署**：在终端启动服务，模拟生产环境运行。
✅ **客户端测试**：使用 `RemoteRunnable` 连接并测试你的 Agent。
✅ **理解持久化机制**：体验基于 Session ID 的状态保存。

---

## ⚙️ Section 1: Setup (环境设置)

### 1.1 切换内核
- 点击右上角的 Kernel (内核) 选择区域。
- 在列表中选择 Python (5-DAY-AGENTS)。

> 关于如何初始化环境和创建内核，请查看 [README 如何运行](../README.md#如何运行)

### 1.2 配置 API Key 和必要的库。

In [1]:
import os
import sys
from dotenv import load_dotenv

# 1. 加载 .env 文件
load_dotenv()

# 2. 配置 API Key
def configure_api_keys():
    # 从环境变量读取配置，如果不存在则使用默认值或留空
    model_name = os.getenv("MODEL_NAME", "gpt-3.5-turbo")
    openai_api_key = os.getenv("OPENAI_API_KEY")
    openai_api_base = os.getenv("OPENAI_API_BASE")

    if not openai_api_key:
        raise ValueError("❌ 未找到 OPENAI_API_KEY，请确保 .env 文件已配置。")

    print(f"✅ API 配置已加载: Model={model_name}, BaseUrl={openai_api_base}")
    return model_name, openai_api_key, openai_api_base

MODEL_NAME, API_KEY, API_BASE = configure_api_keys()

✅ API 配置已加载: Model=Qwen/Qwen3-235B-A22B-Instruct-2507, BaseUrl=https://api-inference.modelscope.cn/v1


## 🏗️ 第二部分：准备 Agent 代码

部署的第一步是将我们的 Agent 逻辑从 Notebook 单元格中提取出来，放入独立的 Python 文件中。这类似于原教程中创建 `sample_agent` 目录的步骤。

我们将创建一个名为 `serving` 的文件夹，并在其中创建两个文件：

1.  `agent.py`: 定义 Agent 的核心逻辑（工具、模型、图）。
2.  `server.py`: 定义 Web 服务器入口，用于启动 API 服务。

In [2]:
# 创建目录
!mkdir -p serving
print("✅ 目录 serving/ 已创建")

✅ 目录 serving/ 已创建


### 2.1 编写 `agent.py`

我们将构建一个简单的 **“天气助手”**。为了演示生产级特性，我们将使用 **LangGraph** 的 `create_react_agent`，因为它内置了对 Checkpointer（记忆持久化）的支持。

这个文件**不运行**服务，只**定义** Agent 对象。

In [3]:
%%writefile serving/agent.py
import os
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# --- 1. 定义工具 ---
@tool
def get_weather(city: str) -> str:
    """
    查询特定城市的天气信息。
    Args:
        city: 城市名称 (e.g., "Tokyo", "New York")
    """
    # 模拟数据
    weather_data = {
        "san francisco": "San Francisco: 晴朗, 22°C",
        "new york": "New York: 多云, 18°C",
        "london": "London: 小雨, 14°C",
        "tokyo": "Tokyo: 晴朗, 21°C",
        "paris": "Paris: 部分多云, 20°C",
        "shanghai": "Shanghai: 阴天, 25°C",
        "beijing": "Beijing: 晴朗, 28°C"
    }
    
    city_lower = city.lower()
    # 简单的模糊匹配
    for k, v in weather_data.items():
        if city_lower in k:
            return v
            
    return f"未找到 '{city}' 的天气信息。"

# --- 2. 配置模型 ---
# 注意：在 server 运行时，环境变量需要已经加载
model = ChatOpenAI(
    model=os.environ.get("MODEL_NAME"),
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    openai_api_base=os.environ.get("OPENAI_API_BASE"),
    temperature=0
)

# --- 3. 创建 Agent ---
# 使用 MemorySaver 来在内存中保存对话状态 (Thread)
# 在生产环境中，你可以替换为 PostgresSaver 或 SqliteSaver 等数据库存储
checkpointer = MemorySaver()

system_message = """
你是一个友好的天气助手。
1. 当用户询问天气时，使用 get_weather 工具。
2. 如果找不到城市，友好地告知用户。
3. 始终用中文回答。
"""

# 这里的 agent 就是一个 CompiledGraph Runnable
agent = create_react_agent(
    model,
    tools=[get_weather],
    prompt=system_message,
    checkpointer=checkpointer,
)

Overwriting serving/agent.py


### 2.2 编写 `server.py`

为了彻底解决 LangServe 与 LangGraph Checkpointer 之间的参数透传问题，我们将使用**原生 FastAPI** 来构建服务。

这种方式让我们能够完全控制请求参数的解析，确保 `thread_id` 能够被正确地传递给 Agent Runtime。这也更接近生产环境中定制化 API 的场景。

In [4]:
%%writefile serving/server.py
import os
import uvicorn
from typing import Any, Dict, List
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage

# 加载环境变量 (确保 API Key 可用)
load_dotenv()

# 导入我们在 agent.py 中定义的 agent
from agent import agent

# 1. 创建 FastAPI 应用
app = FastAPI(
    title="Weather Assistant API",
    version="1.0",
    description="A simple Agent API deployed with custom FastAPI endpoints"
)

# 配置 CORS (允许跨域访问)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 2. 定义请求和响应模型
class InvokeRequest(BaseModel):
    messages: List[Dict[str, str]]
    thread_id: str

class InvokeResponse(BaseModel):
    messages: List[Dict[str, Any]]

# 3. 创建自定义端点
@app.post("/weather/invoke", response_model=InvokeResponse)
async def invoke_agent(request: InvokeRequest):
    """
    调用 Agent 并返回响应。
    这个端点完全控制如何调用 LangGraph agent，确保 config 正确传递。
    """
    try:
        # 构造输入消息
        messages = []
        for msg in request.messages:
            # 支持 human/ai 和 user/assistant 两种格式
            if msg.get("type") == "human" or msg.get("role") == "user":
                messages.append(HumanMessage(content=msg["content"]))
            elif msg.get("type") == "ai" or msg.get("role") == "assistant":
                messages.append(AIMessage(content=msg["content"]))
        
        # 构造 config，包含 thread_id
        config = {
            "configurable": {
                "thread_id": request.thread_id
            }
        }
        
        # 调用 agent (LangGraph CompiledGraph)
        result = await agent.ainvoke(
            {"messages": messages},
            config=config
        )
        
        # 转换响应
        response_messages = []
        for msg in result["messages"]:
            response_messages.append({
                "type": msg.type if hasattr(msg, "type") else "unknown",
                "content": msg.content
            })
        
        return InvokeResponse(messages=response_messages)
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
async def root():
    return {
        "message": "Weather Assistant API",
        "endpoints": {
            "invoke": "/weather/invoke",
            "docs": "/docs"
        }
    }

if __name__ == "__main__":
    # 启动服务，监听 8000 端口
    print("🚀 Starting FastAPI Server on http://localhost:8000")
    print("📝 API Documentation: http://localhost:8000/docs")
    uvicorn.run(app, host="localhost", port=8000)

Overwriting serving/server.py


## ☁️ 第三部分：部署（启动服务）

在真实的云部署中，你会运行 `adk deploy`。在本地模拟中，我们需要运行 `server.py`。

🛑 **重要说明**：

由于 Web 服务器是一个**阻塞进程**（它会一直运行等待请求），我们不能直接在 Notebook 的单元格里运行它，否则 Notebook 会卡住，无法运行后续的测试代码。

**👉 请按照以下步骤操作：**

1.  **打开一个新的终端窗口 (Terminal)**。
2.  **进入项目根目录** (确保你能看到 `.env` 文件)。
3.  **运行以下命令**启动服务：

```bash
# (推荐) 确保加载了依赖环境
python serving/server.py 
# 或者根据你的环境配置使用 uv
# uv run python serving/server.py
```

4.  **等待启动**：直到你看到终端显示 `Starting FastAPI Server` 或 `Uvicorn running on http://localhost:8000`。
5.  **回到这里**：继续运行下面的单元格进行测试。

*(注：如果你无法打开终端，也可以尝试在浏览器访问 `http://localhost:8000/docs` 查看 API 文档)*

## 🤖 第四部分：客户端测试 (使用 Python Requests)

因为我们要测试生产环境 API，最可靠的方法是使用标准的 `requests` 库来模拟客户端调用。这比 `RemoteRunnable` 更透明，更容易调试。

In [5]:
import requests
import json

API_URL = "http://localhost:8000/weather/invoke"

def chat_with_agent(message: str, thread_id: str):
    """发送消息到本地 API 并获取回复"""
    payload = {
        "messages": [
            {"type": "human", "content": message}
        ],
        "thread_id": thread_id
    }
    
    try:
        response = requests.post(API_URL, json=payload)
        response.raise_for_status() # 检查 4xx/5xx 错误
        data = response.json()
        
        # 获取最后一条消息 (AI 回复)
        if data.get("messages"):
            last_msg = data["messages"][-1]
            print(f"🤖 Agent: {last_msg['content']}")
        else:
            print("⚠️ 未收到有效回复")
            
    except requests.exceptions.ConnectionError:
        print("❌ 无法连接到服务器。请确保你已在终端运行 'python serving/server.py'")
    except Exception as e:
        print(f"❌ 请求失败: {e}")
        if 'response' in locals():
            print(f"Server Error Info: {response.text}")

### 4.1 发送测试请求

我们来问问东京的天气。如果一切正常，API 应该返回大模型生成的回复。

In [6]:
print("⏳ 发送请求: 'Tokyo 的天气怎么样？'...")
chat_with_agent("Tokyo 的天气怎么样？", thread_id="test_thread_003")

⏳ 发送请求: 'Tokyo 的天气怎么样？'...
🤖 Agent: 东京的天气晴朗，气温为21°C，非常舒适！如果外出记得做好防晒哦。


## 🧠 第五部分：长时记忆 (Memory Persistence)

现在验证最重要的功能：**记忆**。

我们将使用同一个 `thread_id` 进行两次对话，验证 Agent 是否能记住上下文。

In [7]:
USER_SESSION_ID = "user_jupiter_session_1"

print("\n--- 此轮对话 1 ---")
print("🗣️ 用户: 我叫 Jupiter，我住在 Shanghai。")
chat_with_agent("我叫 Jupiter，我住在 Shanghai。", thread_id=USER_SESSION_ID)


--- 此轮对话 1 ---
🗣️ 用户: 我叫 Jupiter，我住在 Shanghai。
🤖 Agent: 你好，Jupiter！上海是个很棒的城市。需要我帮你查询一下上海的天气吗？


In [8]:
print("\n... 过了很久 (模拟) ...\n")
print("\n--- 此轮对话 2 ---")
print("🗣️ 用户: 我那里的天气怎么样？ (没有提城市名)")

# 我们没有提到 Shanghai，Agent 必须从记忆中通过 USER_SESSION_ID 找回上下文
chat_with_agent("我那里的天气怎么样？", thread_id=USER_SESSION_ID)


... 过了很久 (模拟) ...


--- 此轮对话 2 ---
🗣️ 用户: 我那里的天气怎么样？ (没有提城市名)
🤖 Agent: Jupiter，上海目前是阴天，气温为25°C，天气比较舒适。如果要出门，可以带件薄外套哦！


如果成功，Agent 应该会回答上海的天气，因为它记住了第一轮对话的 Context。

这就是 **Memory Bank** 的本地实现原理：**State Persistence (状态持久化)**。

## 🧹 第六部分：清理资源

在云端，我们需要显式删除 Agent Engine 以防扣费。在本地，我们只需要停止服务器进程。

1.  回到你的终端窗口。
2.  按 `Ctrl + C` 停止服务器。
3.  你可以安全地删除生成的临时文件。

In [ ]:
# 可选：清理生成的文件
# !rm -rf serving
print("✅ 教程结束！")

## 🚀 总结与下一步

恭喜！你完成了 5-Day Agent Course 的所有内容（LangChain 版）！🎉

在今天，你学会了：
1.  使用 `LangGraph` 和 `MemorySaver` 创建带有持久化记忆的生产级 Agent。
2.  使用 `FastAPI` 显式构建 Agent 服务，精确控制参数传递。
3.  在本地模拟“部署-调用”的完整闭环，解决了复杂的配置透传问题。

现在，你可以尝试将你的 Agent 真正部署到服务器上，或者探索更复杂的 LangGraph 流程！

Happy Coding! 🚀